In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !pip install --upgrade albumentations
# !pip install opencv-python-headless==4.1.2.30
# !pip install wandb
# !wandb login

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import to_tensor
from torch.optim import AdamW
import os
import albumentations as A
import numpy as np
from PIL import Image
import wandb

In [ ]:
BATCH_SIZE = 32
EPOCHS = 300
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
class classificationDataset(Dataset):
    def __init__(self, img_dirs, transforms=None):
        self.img_dirs = img_dirs
        self.img_dirs_list = [
            [imgFile for imgFile in os.listdir(files) if imgFile.endswith('jpg')] for files in self.img_dirs
        ]
        self.transforms = transforms
        self.lens = [len(img_dir_list) for img_dir_list in self.img_dirs_list]

    def __len__(self):
        return sum(self.lens)

    def __getitem__(self, idx):
        if idx < self.lens[0]:
            image = Image.open(os.path.join(self.img_dirs[0], self.img_dirs_list[0][idx]))
            image = image.convert('L')
            image = np.array(image)
            if self.transforms:
                image = self.transforms[0](image=image)['image']
            image = to_tensor(image)
            return image, 0
        else:
            image = Image.open(os.path.join(self.img_dirs[1], self.img_dirs_list[1][idx - self.lens[0]]))
            image = image.convert('L')
            image = np.array(image)
            if self.transforms:
                image = self.transforms[1](image=image)['image']
            image = to_tensor(image)
            return image, 1

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq = nn.Sequential(
            self._block(1, 32, 3),
            self._block(32, 48, 3, 2),
            self._block(48, 64, 3),
            self._block(64, 80, 3),
            self._block(80, 96, 3, 2),
            self._block(96, 112, 3),
            self._block(112, 128, 3),
            self._block(128, 144, 3, 2),
            self._block(144, 154, 3),
            self._block(154, 116, 3),
            Flatten(),
            nn.Linear(16704, 2, bias=False),
            nn.BatchNorm1d(2), 
        ) 

    def _block(self, input_dim, output_dim, kernel_size, stride=1):
        return nn.Sequential(
            nn.Conv2d(input_dim, output_dim, kernel_size, stride, bias=False),
            nn.BatchNorm2d(output_dim),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.seq(x)
        return F.log_softmax(x, dim=1)

class Flatten(nn.Module):
    def forward(self, x):
        return torch.flatten(x.permute(0, 2, 3, 1), 1)

In [ ]:
model = Model().to(device)
optimizer = AdamW(model.parameters())

tictactoeTransform = A.Compose([
    A.augmentations.geometric.transforms.Perspective(p=1, scale=(0.005, 0.02)),
    A.augmentations.transforms.OpticalDistortion(p=0.2),
    A.augmentations.geometric.transforms.ElasticTransform(p=1, alpha=1, sigma=4, alpha_affine=4),
    A.RandomCrop(160, 160, p=0.25),
    A.InvertImg(p=0.1),
    A.ColorJitter(brightness=0.55, contrast=0.6, saturation=0.6, hue=0.6, p=0.4),
    A.GaussNoise(p=0.12),
    A.Blur(blur_limit=3, p=0.22),
    A.GlassBlur(max_delta=1, iterations=1, p=0.14),
    A.CLAHE(p=0.22, tile_grid_size=(4, 4)),
    A.Sharpen(p=0.18, alpha=0.2, lightness=1.5),
    A.Emboss(p=0.18),
    A.Equalize(p=0.04),
    A.MultiplicativeNoise(p=0.22),
    A.Resize(168, 168, p=1.0, interpolation=Image.NEAREST),
    A.RandomBrightness(p=0.22),
    A.RandomContrast(p=0.22),
    A.RandomGamma(p=0.22),
    A.Solarize(threshold=128, p=0.2),
])

connectfourTransform = A.Compose([
    A.augmentations.geometric.transforms.Perspective(p=1, scale=(0.005, 0.02)),
    A.augmentations.transforms.OpticalDistortion(p=0.2),
    A.augmentations.transforms.OpticalDistortion(p=0.2),
    A.augmentations.geometric.transforms.ElasticTransform(p=0.2, alpha=1, sigma=0.5, alpha_affine=0.5),
    A.RandomCrop(154, 154, p=0.25),
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(brightness=0.55, contrast=0.6, saturation=0.6, hue=0.6, p=0.4),
    A.GaussNoise(p=0.12),
    A.Blur(blur_limit=3, p=0.22),
    A.GlassBlur(max_delta=1, iterations=1, p=0.14),
    A.CLAHE(p=0.22, tile_grid_size=(4, 4)),
    A.Sharpen(p=0.18, alpha=0.2, lightness=1.5),
    A.Emboss(p=0.18),
    A.Equalize(p=0.04),
    A.MultiplicativeNoise(p=0.22),
    A.Resize(168, 168, p=1.0, interpolation=Image.NEAREST),
    A.RandomBrightness(p=0.22),
    A.RandomContrast(p=0.22),
    A.RandomGamma(p=0.22),
    A.Solarize(threshold=128, p=0.2),
])

transforms = [tictactoeTransform, connectfourTransform]

dataset = classificationDataset(['drive/MyDrive/boardsTTT', 'drive/MyDrive/boardsC4'], transforms=transforms)

class_weights = [1/1064, 1/1606]
sample_weights = [0] * len(dataset)

for idx, (data, label) in enumerate(dataset):
    class_weight = class_weights[label]
    sample_weights[idx] = class_weight

sampler = torch.utils.data.sampler.WeightedRandomSampler(sample_weights, num_samples=len(sample_weights))

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=sampler)

In [ ]:
def main():
    wandb.init(project="boardGameClassification", entity="robertfoerster")
    for epoch in range(EPOCHS):
        for idx, (value, label) in enumerate(dataloader):
            value, label = value.to(device), label.to(device)
            output = model(value)
            loss = F.nll_loss(output, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            metrics = {
                'train/train_loss': loss,
                'train/epoch': epoch,
            }
            wandb.log(metrics)
    wandb.finish()

In [ ]:
main()

In [ ]:
torch.save(model.state_dict(), 'drive/MyDrive/classificaton.pth')
torch.save(optimizer.state_dict(), 'drive/MyDrive/clfOptim.pth')